**MODEL: Convolution Autoencoder -> LSTM**

**Import Statements and Global Variables**

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
import os
from keras import backend as K
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPooling2D, Flatten, Dense,TimeDistributed, UpSampling2D
from keras.models import Model, load_model
from keras import metrics 
import random
import pickle

# Directory paths
VIDEOS_DIR = './Videos/'
IMAGES_DIR = './Images/'

**Labels for classes**

In [27]:
# List of all classes
classes = ['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 
           'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']

# Create dictionary to label each class
class_to_index = {}
for i in range(len(classes)):
    class_to_index[classes[i]] = i

**Standard Function Definitions**

In [28]:
def permute(X,Y):
    train_size = X.shape[0]
    permutation_train = np.random.permutation(train_size)
    X = X[permutation_train]
    Y = Y[permutation_train]
    return X,Y

def load_image(path,image_size):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    return image

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def pad(X_cnn,max_len):
    features_len = np.shape(X_cnn)[1]
    length = np.shape(X_cnn)[0]
    X_cnn = list(X_cnn)
    pad_arr = [0 for i in range(features_len)]
    for i in range(max_len-length):
        X_cnn.append(pad_arr)
    return np.array(X_cnn)

def evaluate(X_test,Y_test,model):
    count = 0
    for i in range(len(X_test)):
        pred = model.predict(X_test[i])
        max_pred = [np.argmax(i) for i in pred]
        counts = np.bincount(max_pred)
        class_pred = np.argmax(counts)
        #class_pred = max_pred
        actual = np.argmax(Y_test[i])
        #print("Max Preds time", max_pred)
        #print("Pred",classes[class_pred],"Actual",classes[actual])
        #print()
        if class_pred == actual:
            count += 1
    return float(count)/float(len(Y_test)) * 100.0

**Reading Data**

In [29]:
# List of directories in Videos folder
videos = []
for x in classes:
    videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
videos

def build_dataset(image_size, images_per_video = 10):
    # Initialise image vectors
    X_train_images = []
    Y_train_images = []
    
    # For each class in classes, do:
    vid_num, l_vid_num = 0, []
    for i in range(len(classes)):
        # Name of class
        cls = classes[i]
        
        print ("Processing ", cls, " videos...")
        # For each dir in corresponding videos folder, do:
        for j in range(len(videos[i])):
            # Get dir no, eg. '012', '016', etc.
            vid = videos[i][j]
            print ("Going through ", vid, "...")
            
            # Append to new paths
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            
            # List all images in the given image directory
            filelist = list(os.listdir(image_r))
            #print(sorted(filelist))
            
            # Initialise X_train
            X_train_images_video = []
            
            # For image in imagelist, do:
            for fichier in filelist:
                # Check if file is an image
                if fichier.endswith(".png"):
                    # Load and resize image
                    image = load_image(image_r+fichier,image_size)
                    # Append to list
                    X_train_images_video.append(image)
                    
            # Randomly shuffle list
            permutation = np.random.permutation(len(X_train_images_video))
            X_train_images_video = np.array(X_train_images_video)[permutation]
            
            # If no. of images in video < images_per_video, take that
            no_of_images = min(images_per_video, len(X_train_images_video))
            
            # Append to X_train images
            X_train_images += list(X_train_images_video[:no_of_images])
            
            # Append to Y_train_images
            Y_train_images += [i] * no_of_images
            
            # To keep track of video number
            l_vid_num += [vid_num] * no_of_images
            vid_num += 1
            
    print ("Done.")
    return np.array(X_train_images), np.array(Y_train_images), l_vid_num, vid_num

**Encoder and LSTM Models**

In [30]:
def build_encoder(input_dim):
    input_img = Input(shape=input_dim)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(input_img, decoded)
    encoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    return encoder

In [31]:
def LSTM_model(encoding_dim):
    X_input = Input(shape=encoding_dim)
    X = LSTM(16, return_sequences=True)(X_input)
    X = Dropout(0.3)(X)
    X = LSTM(16, return_sequences=False)(X)
    X = Dropout(0.3)(X)
    X = Dense(9,activation='softmax')(X)
    
    lstm = Model(X_input, X)
    lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    return lstm

**Read Data and Build Models**

In [32]:
data = build_dataset((172, 172))

Processing  Kicking  videos...
Going through  012 ...
Going through  016 ...
Going through  011 ...
Going through  017 ...
Going through  013 ...
Going through  003 ...
Going through  009 ...
Going through  015 ...
Going through  006 ...
Going through  007 ...
Going through  004 ...
Going through  002 ...
Going through  014 ...
Going through  001 ...
Going through  010 ...
Going through  005 ...
Processing  Riding-Horse  videos...
Going through  003 ...
Going through  009 ...
Going through  006 ...
Going through  007 ...
Going through  004 ...
Going through  002 ...
Going through  008 ...
Going through  001 ...
Going through  010 ...
Going through  005 ...
Processing  Running  videos...
Going through  009 ...
Going through  006 ...
Going through  007 ...
Going through  004 ...
Going through  002 ...
Going through  008 ...
Going through  001 ...
Going through  010 ...
Going through  005 ...
Processing  SkateBoarding  videos...
Going through  003 ...
Going through  009 ...
Going through 

In [33]:
x_img, y_img, lvid, no_of_videos = data

In [59]:
encoder = build_encoder((3, 172, 172))
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 3, 172, 172)       0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 16, 172, 172)      448       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 16, 86, 86)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 8, 86, 86)         1160      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 8, 43, 43)         0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 8, 43, 43)         584       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 8, 22, 22)         0         
__________

In [55]:
lstm = LSTM_model((10, 968))
lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 10, 968)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 10, 16)            63040     
_________________________________________________________________
dropout_13 (Dropout)         (None, 10, 16)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 16)                2112      
_________________________________________________________________
dropout_14 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 9)                 153       
Total params: 65,305
Trainable params: 65,305
Non-trainable params: 0
_________________________________________________________________


**Train Models**

In [36]:
np.shape(x_img)

(1097, 172, 172, 3)

In [37]:
x_img = x_img.astype('float32') / 255.
x_img = np.reshape(x_img, (len(x_img), 3, 172, 172))

In [60]:
encoder.fit(x_img, x_img,
                epochs=20,
                batch_size=32,
                shuffle=True,
                validation_split=0.2)

Train on 877 samples, validate on 220 samples
Epoch 1/20
877/877 [==============================] - 144s 164ms/step - loss: 0.6704 - val_loss: 0.6413
Epoch 2/20
877/877 [==============================] - 141s 161ms/step - loss: 0.6329 - val_loss: 0.6594
Epoch 3/20
877/877 [==============================] - 126s 144ms/step - loss: 0.6238 - val_loss: 0.6234
Epoch 4/20
877/877 [==============================] - 128s 146ms/step - loss: 0.6164 - val_loss: 0.6233
Epoch 5/20
877/877 [==============================] - 175s 200ms/step - loss: 0.6142 - val_loss: 0.6303
Epoch 6/20
877/877 [==============================] - 147s 168ms/step - loss: 0.6136 - val_loss: 0.6177
Epoch 7/20
877/877 [==============================] - 151s 172ms/step - loss: 0.6126 - val_loss: 0.6264
Epoch 8/20
877/877 [==============================] - 167s 191ms/step - loss: 0.6102 - val_loss: 0.6172
Epoch 9/20
877/877 [==============================] - 160s 183ms/step - loss: 0.6100 - val_loss: 0.6153
Epoch 10/20
877/87

In [ ]:
get_8th_layer_output = K.function([encoder.layers[0].input],
                                  [encoder.layers[8].output])

layer_output = get_8th_layer_output([x_img])[0]

In [41]:
np.shape(layer_output)

(1097, 8, 11, 11)

In [42]:
lstm_in = [None] * no_of_videos
for i in range(len(lstm_in)):
    lstm_in[i] = []

for i in range(len(layer_output)):
    img = np.reshape(layer_output[i], -1)
    lstm_in[lvid[i]].append(img)

print (np.shape(lstm_in))
    
lens = [len(a) for a in lstm_in]
max_len = max(lens)
print (max_len)

for i in range(len(lstm_in)):
    lstm_in[i] = pad(lstm_in[i], max_len)
    
for a in lstm_in:
    print (np.shape(a))
    
lstm_in = np.array(lstm_in)

(111,)
10
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)
(10, 968)


In [43]:
np.shape(lstm_in)

(111, 10, 968)

In [44]:
lstm_y = convert_to_one_hot(y_img, 9)
np.shape(lstm_y)

(1097, 9)

In [45]:
lstm_y = [ np.where(r==1)[0][0] for r in lstm_y]

In [46]:
lstm_y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [47]:
y = np.array([None] * no_of_videos)
    
for i in range(len(lstm_y)):
    y[lvid[i]] = lstm_y[i]
    
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
       5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8], dtype=object)

In [48]:
for i in range(len(y)):
    y[i] = int(y[i])

y = np.array(y, dtype=np.int8)
    
y = convert_to_one_hot(y, 9)
y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0.

In [56]:
lstm.fit(lstm_in, y, epochs=400, batch_size=128, shuffle=True, validation_split=0.2)

Train on 88 samples, validate on 23 samples
Epoch 1/400
88/88 [==============================] - 2s 26ms/step - loss: 0.3502 - acc: 0.8889 - val_loss: 0.3702 - val_acc: 0.8889
Epoch 2/400
88/88 [==============================] - 0s 880us/step - loss: 0.3446 - acc: 0.8889 - val_loss: 0.3811 - val_acc: 0.8889
Epoch 3/400
88/88 [==============================] - 0s 749us/step - loss: 0.3433 - acc: 0.8889 - val_loss: 0.3874 - val_acc: 0.8889
Epoch 4/400
88/88 [==============================] - 0s 832us/step - loss: 0.3442 - acc: 0.8889 - val_loss: 0.3920 - val_acc: 0.8889
Epoch 5/400
88/88 [==============================] - 0s 975us/step - loss: 0.3424 - acc: 0.8889 - val_loss: 0.3952 - val_acc: 0.8889
Epoch 6/400
88/88 [==============================] - 0s 1ms/step - loss: 0.3396 - acc: 0.8889 - val_loss: 0.3987 - val_acc: 0.8889
Epoch 7/400
88/88 [==============================] - 0s 1ms/step - loss: 0.3418 - acc: 0.8889 - val_loss: 0.4006 - val_acc: 0.8889
Epoch 8/400
88/88 [===========

Epoch 63/400
88/88 [==============================] - 0s 1ms/step - loss: 0.2905 - acc: 0.8902 - val_loss: 0.4809 - val_acc: 0.8889
Epoch 64/400
88/88 [==============================] - 0s 896us/step - loss: 0.3001 - acc: 0.8939 - val_loss: 0.5247 - val_acc: 0.8889
Epoch 65/400
88/88 [==============================] - 0s 802us/step - loss: 0.3023 - acc: 0.8927 - val_loss: 0.5294 - val_acc: 0.8889
Epoch 66/400
88/88 [==============================] - 0s 782us/step - loss: 0.2992 - acc: 0.8914 - val_loss: 0.4992 - val_acc: 0.8889
Epoch 67/400
88/88 [==============================] - 0s 765us/step - loss: 0.2919 - acc: 0.8902 - val_loss: 0.4384 - val_acc: 0.8889
Epoch 68/400
88/88 [==============================] - 0s 790us/step - loss: 0.3165 - acc: 0.8902 - val_loss: 0.4393 - val_acc: 0.8889
Epoch 69/400
88/88 [==============================] - 0s 869us/step - loss: 0.3015 - acc: 0.8914 - val_loss: 0.4999 - val_acc: 0.8889
Epoch 70/400
88/88 [==============================] - 0s 913us/s

Epoch 125/400
88/88 [==============================] - 0s 914us/step - loss: 0.2643 - acc: 0.9015 - val_loss: 0.5354 - val_acc: 0.8599
Epoch 126/400
88/88 [==============================] - 0s 887us/step - loss: 0.2604 - acc: 0.9040 - val_loss: 0.5595 - val_acc: 0.8792
Epoch 127/400
88/88 [==============================] - 0s 797us/step - loss: 0.2765 - acc: 0.8965 - val_loss: 0.5539 - val_acc: 0.8841
Epoch 128/400
88/88 [==============================] - 0s 792us/step - loss: 0.2746 - acc: 0.8939 - val_loss: 0.5305 - val_acc: 0.8841
Epoch 129/400
88/88 [==============================] - 0s 956us/step - loss: 0.2188 - acc: 0.9091 - val_loss: 0.4792 - val_acc: 0.8696
Epoch 130/400
88/88 [==============================] - 0s 898us/step - loss: 0.2769 - acc: 0.8889 - val_loss: 0.5016 - val_acc: 0.8744
Epoch 131/400
88/88 [==============================] - 0s 924us/step - loss: 0.2231 - acc: 0.9129 - val_loss: 0.5408 - val_acc: 0.8841
Epoch 132/400
88/88 [==============================] - 

Epoch 187/400
88/88 [==============================] - 0s 819us/step - loss: 0.1872 - acc: 0.9230 - val_loss: 0.5599 - val_acc: 0.8502
Epoch 188/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1925 - acc: 0.9242 - val_loss: 0.5976 - val_acc: 0.8502
Epoch 189/400
88/88 [==============================] - 0s 990us/step - loss: 0.2221 - acc: 0.9053 - val_loss: 0.5544 - val_acc: 0.8599
Epoch 190/400
88/88 [==============================] - 0s 935us/step - loss: 0.1817 - acc: 0.9268 - val_loss: 0.5120 - val_acc: 0.8647
Epoch 191/400
88/88 [==============================] - 0s 1ms/step - loss: 0.2226 - acc: 0.9066 - val_loss: 0.5441 - val_acc: 0.8696
Epoch 192/400
88/88 [==============================] - 0s 985us/step - loss: 0.1980 - acc: 0.9217 - val_loss: 0.6026 - val_acc: 0.8454
Epoch 193/400
88/88 [==============================] - 0s 840us/step - loss: 0.2216 - acc: 0.9129 - val_loss: 0.5835 - val_acc: 0.8599
Epoch 194/400
88/88 [==============================] - 0s 8

Epoch 249/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1555 - acc: 0.9394 - val_loss: 0.5958 - val_acc: 0.8502
Epoch 250/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1449 - acc: 0.9444 - val_loss: 0.5747 - val_acc: 0.8647
Epoch 251/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1659 - acc: 0.9306 - val_loss: 0.6010 - val_acc: 0.8647
Epoch 252/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1564 - acc: 0.9419 - val_loss: 0.6204 - val_acc: 0.8599
Epoch 253/400
88/88 [==============================] - 0s 943us/step - loss: 0.1539 - acc: 0.9369 - val_loss: 0.6017 - val_acc: 0.8406
Epoch 254/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1549 - acc: 0.9381 - val_loss: 0.5754 - val_acc: 0.8357
Epoch 255/400
88/88 [==============================] - 0s 965us/step - loss: 0.1581 - acc: 0.9432 - val_loss: 0.6167 - val_acc: 0.8357
Epoch 256/400
88/88 [==============================] - 0s 1ms/ste

Epoch 311/400
88/88 [==============================] - 0s 962us/step - loss: 0.1295 - acc: 0.9495 - val_loss: 0.5925 - val_acc: 0.8551
Epoch 312/400
88/88 [==============================] - 0s 794us/step - loss: 0.1401 - acc: 0.9444 - val_loss: 0.6107 - val_acc: 0.8551
Epoch 313/400
88/88 [==============================] - 0s 801us/step - loss: 0.1535 - acc: 0.9407 - val_loss: 0.6611 - val_acc: 0.8502
Epoch 314/400
88/88 [==============================] - 0s 772us/step - loss: 0.1325 - acc: 0.9432 - val_loss: 0.6731 - val_acc: 0.8213
Epoch 315/400
88/88 [==============================] - 0s 842us/step - loss: 0.1496 - acc: 0.9432 - val_loss: 0.6329 - val_acc: 0.8309
Epoch 316/400
88/88 [==============================] - 0s 807us/step - loss: 0.1516 - acc: 0.9470 - val_loss: 0.6733 - val_acc: 0.8309
Epoch 317/400
88/88 [==============================] - 0s 788us/step - loss: 0.1180 - acc: 0.9621 - val_loss: 0.7083 - val_acc: 0.8406
Epoch 318/400
88/88 [==============================] - 

88/88 [==============================] - 0s 921us/step - loss: 0.1269 - acc: 0.9520 - val_loss: 0.6757 - val_acc: 0.8261
Epoch 373/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1126 - acc: 0.9545 - val_loss: 0.7286 - val_acc: 0.8261
Epoch 374/400
88/88 [==============================] - 0s 939us/step - loss: 0.1176 - acc: 0.9558 - val_loss: 0.7158 - val_acc: 0.8309
Epoch 375/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1116 - acc: 0.9533 - val_loss: 0.6593 - val_acc: 0.8164
Epoch 376/400
88/88 [==============================] - 0s 908us/step - loss: 0.1152 - acc: 0.9533 - val_loss: 0.7091 - val_acc: 0.8261
Epoch 377/400
88/88 [==============================] - 0s 980us/step - loss: 0.1097 - acc: 0.9558 - val_loss: 0.7059 - val_acc: 0.8357
Epoch 378/400
88/88 [==============================] - 0s 1ms/step - loss: 0.1123 - acc: 0.9520 - val_loss: 0.6869 - val_acc: 0.8261
Epoch 379/400
88/88 [==============================] - 0s 1ms/step - loss: 

**Save Model**

In [56]:
# Save model here